<a href="https://colab.research.google.com/github/I-am-vishalmaurya/Covid_vaccination_progress_in-World-and-India/blob/gh-pages/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
!nvidia-smi

Tue Apr 27 06:34:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
import keras
from tensorflow.keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import *
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [10]:
data = pd.read_csv("/content/drive/MyDrive/Data Science/IMDB_movie_review/IMDB Dataset.csv")

In [11]:
def remove_tags(strings):
  result = re.sub('<.*?>','', strings)
  return result
data['review'] = data['review'].apply(lambda x: x.lower())
data['review'] = data['review'].apply(lambda x: remove_tags(x))

In [12]:
data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [13]:
print("Postivie review in data")
print(data[ data['sentiment'] == 'positive'].count())
print("Negative review in data")
print(data[ data['sentiment'] == 'negative'].count())

Postivie review in data
review       25000
sentiment    25000
dtype: int64
Negative review in data
review       25000
sentiment    25000
dtype: int64


In [14]:
max_features = 1000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['review'].values)
X = tokenizer.texts_to_sequences(data['review'].values)
X = pad_sequences(X)

In [15]:
import pickle
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [16]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(33500, 1727) (33500, 2)
(16500, 1727) (16500, 2)


In [17]:
dropout_rate = 0.3
batch_size = 1000
activation_func = keras.activations.relu

schema = [
          Embedding(5000, 10, input_length=120),
          LSTM(32),
          Dropout(dropout_rate),
          Dense(32, activation=activation_func),
          Dropout(dropout_rate),
          Dense(2 , activation=keras.activations.softmax)
]

model = keras.Sequential(schema)
model.compile(
    optimizer=keras.optimizers.Adam() ,
    loss=keras.losses.categorical_crossentropy ,
    metrics=[ 'accuracy']
)

In [18]:
model.fit(X_train,Y_train,batch_size=1000, epochs=8)

Epoch 1/8
34/34 [==============================] - 23s 176ms/step - loss: 0.6915 - accuracy: 0.5309
Epoch 2/8
34/34 [==============================] - 6s 175ms/step - loss: 0.5971 - accuracy: 0.7269
Epoch 3/8
34/34 [==============================] - 6s 175ms/step - loss: 0.4257 - accuracy: 0.8185
Epoch 4/8
34/34 [==============================] - 6s 174ms/step - loss: 0.3444 - accuracy: 0.8600
Epoch 5/8
34/34 [==============================] - 6s 173ms/step - loss: 0.3240 - accuracy: 0.8699
Epoch 6/8
34/34 [==============================] - 6s 174ms/step - loss: 0.3131 - accuracy: 0.8733
Epoch 7/8
34/34 [==============================] - 6s 176ms/step - loss: 0.3009 - accuracy: 0.8817
Epoch 8/8
34/34 [==============================] - 6s 175ms/step - loss: 0.3064 - accuracy: 0.8784


In [19]:
model.evaluate( X_test , Y_test ) 

516/516 [==============================] - 14s 27ms/step - loss: 0.3071 - accuracy: 0.8716


[0.30711665749549866, 0.8716363906860352]

In [21]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [22]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [23]:
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

accuracy: 87.16%


In [26]:
twt = ['That movie was so good and everyone perform much  better.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0  11  16  12  33  48   2 303  71 125]]
1/1 - 0s
positive
